In [20]:
from symxplorer.spice_engine.spicelib import LTspice_Wrapper

from symxplorer.designer_tools.bayesian_ax import Ax_LTspice_Bode_Fitter

from symxplorer.designer_tools.tf_models import Second_Order_LP_TF, First_Order_LP_TF, cascade_tf

from symxplorer.designer_tools.utils import Frequency_Weight, plot_complex_response

In [21]:
ltspice_wrapper = LTspice_Wrapper(asc_filename="asc/TIA-ALL_Z-LPF-AC.asc", 
                          traces_of_interest=['frequency', 'Vout'],
                          dump_parent_folder="runner")

2025-02-09 17:33:08,344 - spicelib.AscEditor - Parsing ASC file asc/TIA-ALL_Z-LPF-AC.asc
2025-02-09 17:33:08,351 - spicelib.AsyReader - Parsing ASY file /Users/danialnoorizadeh/.wine/drive_c/users/danialnoorizadeh/AppData/Local/LTspice/lib/sym/voltage.asy
2025-02-09 17:33:08,352 - spicelib.AsyReader - Parsing ASY file /Users/danialnoorizadeh/.wine/drive_c/users/danialnoorizadeh/AppData/Local/LTspice/lib/sym/voltage.asy
2025-02-09 17:33:08,359 - spicelib.AsyReader - Parsing ASY file /Users/danialnoorizadeh/.wine/drive_c/users/danialnoorizadeh/AppData/Local/LTspice/lib/sym/OpAmps/LT1818.asy
2025-02-09 17:33:08,361 - spicelib.Utils - Searching for 'LTC2.lib' in 'asc'
2025-02-09 17:33:08,362 - spicelib.Utils - Searching for 'LTC2.lib' in '.'
2025-02-09 17:33:08,365 - spicelib.Utils - Searching for 'LTC2.lib' in '/Users/danialnoorizadeh/.wine/drive_c/users/danialnoorizadeh/AppData/Local/LTspice/lib/sub'
2025-02-09 17:33:08,386 - spicelib.Utils - Found '/Users/danialnoorizadeh/.wine/drive_c/

In [22]:
ltspice_wrapper.cap_unit, ltspice_wrapper.res_unit

('p', 'k')

In [23]:
# Butterworth Filter (3rd order)
fc = 1e7
tf1 = Second_Order_LP_TF(q=1, fc=1.272*fc, dc_gain=1e3)
tf2 = First_Order_LP_TF(fc=1*fc, dc_gain=1)

target_tf = cascade_tf(list_of_tfs=[tf1, tf2], dc_gain_multiplier=1)

target_tf

1000.0/((5.0e-8*s/pi + 1)*(1.54513270835806e-15*s**2/pi**2 + 3.93081761006289e-8*s/pi + 1))

In [24]:
frequency_weights = Frequency_Weight(lower= 0.1*fc, upper= 10*fc, bias=10)
# We dont know the frequency until we run the simulation
# frequency_weights.compute_weights()

In [ ]:
optimizer = Ax_LTspice_Bode_Fitter(
    ltspice_wrapper=ltspice_wrapper, 
    target_tf=target_tf,
    design_params= { # -1 is a dummy bound 
        "C"  : -1,
        "R1" : -1,
        "R2" : -1,
        "R3" : -1,
        "R5" : -1,
        "R6" : -1,
    },
    c_range= [1, 1e3],
    r_range=[1e-2, 1e2],
    output_node="V(vout)",
    frequency_weight = frequency_weights,
    max_loss=50,
    loss_fn="mse",
    norm_method="min-max",
    random_seed=42,
    )

In [ ]:
optimizer.parameterize()

In [ ]:
optimizer.create_experiment(num_sobol_trials=5)

In [ ]:
optimizer.optimization_loop(num_trials=100, include_mag_loss=True, include_phase_loss=True)

In [ ]:
from ax.utils.notebook.plotting import init_notebook_plotting, render
render(optimizer.ax_client.get_optimization_trace(objective_optimum = 0))

In [ ]:
idx = 72
complex_response = optimizer.optimization_log[idx]["complex_response"]
optimizer.optimization_log[idx]["bode_fitting_loss"], optimizer.optimization_log[idx]["params"]


In [ ]:
optimizer.optimization_log[idx]["mag_loss"], optimizer.optimization_log[idx]["phase_loss"]

In [ ]:
frequency_weights.weights

In [ ]:
plot_complex_response(optimizer.frequency_array, [complex_response, optimizer.target_complex_response], ["optimized", "Target"])

In [ ]:

render(optimizer.ax_client.get_contour_plot())

In [ ]:
optimizer.frequency_weight.lower, optimizer.frequency_weight.upper